In [1]:
import os
import zipfile
import io
import csv
import pandas as pd
from gdc.utils import data_path, ExtendedNamespace

In [2]:
CMS_DATA_PATH = data_path('cms_data.txt')
CMS_DATA_PATH

'/home/sylvain/Dropbox/Econ/papiers/gameTheory/generous_dynamic_contracting/datasets/us'

In [3]:
def get_zip_files():
    zip_files = [os.path.join(CMS_DATA_PATH, f) 
                 for f in os.listdir(CMS_DATA_PATH)
                 if f.lower().endswith(".zip")]
    return ExtendedNamespace(**{z.split('/')[-1].rstrip('.zip') : z for z in zip_files})
    
    
ZF = get_zip_files()

In [4]:
def get_zipfile_metrics(zip_path):
    """
    Return (column_names, num_columns, num_rows) for a CSV inside a ZIP.
    Counts rows via streamed iteration (safe for very large files).
    """
    with zipfile.ZipFile(zip_path, "r") as z:
        csv_name = next(n for n in z.namelist() if n.lower().endswith(".csv"))
        
        with z.open(csv_name, "r") as f:
            text_stream = io.TextIOWrapper(f, encoding="utf-8", newline="")
            reader = csv.reader(text_stream)
            
            # Read header
            try:
                header = next(reader)
            except StopIteration:
                return [], 0, 0
            
            num_cols = len(header)
            
            # Count rows line-by-line
            num_rows = sum(1 for _ in reader)
            
    return ExtendedNamespace(**{'columns': header, 'num_cols': num_cols, 'num_rows': num_rows})

get_zipfile_metrics(ZF.DE1_0_2008_Beneficiary_Summary_File_Sample_1)

namespace(columns=['DESYNPUF_ID',
                   'BENE_BIRTH_DT',
                   'BENE_DEATH_DT',
                   'BENE_SEX_IDENT_CD',
                   'BENE_RACE_CD',
                   'BENE_ESRD_IND',
                   'SP_STATE_CODE',
                   'BENE_COUNTY_CD',
                   'BENE_HI_CVRAGE_TOT_MONS',
                   'BENE_SMI_CVRAGE_TOT_MONS',
                   'BENE_HMO_CVRAGE_TOT_MONS',
                   'PLAN_CVRG_MOS_NUM',
                   'SP_ALZHDMTA',
                   'SP_CHF',
                   'SP_CHRNKIDN',
                   'SP_CNCR',
                   'SP_COPD',
                   'SP_DEPRESSN',
                   'SP_DIABETES',
                   'SP_ISCHMCHT',
                   'SP_OSTEOPRS',
                   'SP_RA_OA',
                   'SP_STRKETIA',
                   'MEDREIMB_IP',
                   'BENRES_IP',
                   'PPPYMT_IP',
                   'MEDREIMB_OP',
                   'BENRES_OP',
        

In [5]:
def zip_chunk_generator(zip_path, batch_size=None, max_batches=None, usecols=None):
    zip_path = os.path.join(CMS_DATA_PATH, zip_path)
    with zipfile.ZipFile(zip_path) as z:
        # detect inner CSV file
        csv_name = next(n for n in z.namelist() if n.lower().endswith(".csv"))

        with z.open(csv_name) as f:
            # wrap raw bytes → text
            text_stream = io.TextIOWrapper(f, encoding="utf-8", newline="")

            if batch_size is None:
                df = pd.read_csv(
                    text_stream,
                    low_memory=False,
                    usecols=usecols,
                )
                yield df
                return


            # streamed chunk reader
            reader = pd.read_csv(
                text_stream,
                chunksize=batch_size,
                low_memory=False,
                usecols=usecols,
            )

            for i, chunk in enumerate(reader):
                yield chunk

                if max_batches is not None and (i + 1) >= max_batches:
                    break

chunks = zip_chunk_generator(ZF.DE1_0_2008_Beneficiary_Summary_File_Sample_1, 10, 1)

In [6]:
next(chunks)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
0,00013D2EFD8E45D1,19230501,NaN,1,1,0,26,950,12,12,...,2,0.0,0.0,0.0,50.0,10.0,0.0,0.0,0.0,0.0
1,00016F745862898F,19430101,NaN,1,1,0,39,230,12,12,...,2,0.0,0.0,0.0,0.0,0.0,0.0,700.0,240.0,0.0
2,0001FDD721E223DC,19360901,NaN,2,1,0,39,280,12,12,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00021CA6FF03E670,19410601,NaN,1,5,0,6,290,0,0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00024B3D2352D2D0,19360801,NaN,1,1,0,52,590,12,12,...,2,0.0,0.0,0.0,30.0,40.0,0.0,220.0,80.0,0.0
5,0002DAE1C81CC70D,19431001,NaN,1,2,0,33,400,0,0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0002F28CE057345B,19220701,NaN,1,1,0,39,270,12,12,...,2,0.0,0.0,0.0,1010.0,270.0,0.0,3330.0,940.0,0.0
7,000308435E3E5B76,19350901,NaN,1,1,0,24,680,10,10,...,2,0.0,0.0,0.0,150.0,160.0,0.0,870.0,340.0,80.0
8,000345A39D4157C9,19760901,NaN,2,1,0,23,810,0,0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,00036A21B65B0206,19381001,NaN,2,2,0,1,570,12,12,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_beneficiaries_2008 = next(zip_chunk_generator(ZF.DE1_0_2008_Beneficiary_Summary_File_Sample_1))
df_beneficiaries_2008['Year'] = 2008

In [8]:
df_beneficiaries_2009 = next(zip_chunk_generator(ZF.DE1_0_2009_Beneficiary_Summary_File_Sample_1))
df_beneficiaries_2009['Year'] = 2009

In [9]:
df_beneficiaries_2010 = next(zip_chunk_generator(ZF.DE1_0_2010_Beneficiary_Summary_File_Sample_1))
df_beneficiaries_2010['Year'] = 2010

In [10]:
def load_codebook():
    df = pd.read_csv(os.path.join(CMS_DATA_PATH, 'code_book.csv'))

    mapping = dict(zip(df["variable"], df["name"]))
    return ExtendedNamespace(**mapping)

In [11]:
CB = load_codebook()

In [12]:
CB.__dict__

{'patient_id': 'DESYNPUF_ID',
 'date_birth': 'BENE_BIRTH_DT',
 'date_death': 'BENE_DEATH_DT',
 'sex': 'BENE_SEX_IDENT_CD',
 'race': 'BENE_RACE_CD',
 'cc_esrd_indicator': 'BENE_ESRD_IND',
 'state_code': 'SP_STATE_CODE',
 'county_code': 'BENE_COUNTY_CD',
 'months_partA': 'BENE_HI_CVRAGE_TOT_MONS',
 'months_partB': 'BENE_SMI_CVRAGE_TOT_MONS',
 'months_hmo': 'BENE_HMO_CVRAGE_TOT_MONS',
 'months_partD': 'PLAN_CVRG_MOS_NUM',
 'cc_alzheimer': 'SP_ALZHDMTA',
 'cc_chf': 'SP_CHF',
 'cc_chronic_kidney': 'SP_CHRNKIDN',
 'cc_cancer': 'SP_CNCR',
 'cc_copd': 'SP_COPD',
 'cc_depression': 'SP_DEPRESSN',
 'cc_diabetes': 'SP_DIABETES',
 'cc_ischemic_heart': 'SP_ISCHMCHT',
 'cc_osteoporosis': 'SP_OSTEOPRS',
 'cc_ra_oa': 'SP_RA_OA',
 'cc_stroke_tia': 'SP_STRKETIA',
 'medicare_reimb_ip': 'MEDREIMB_IP',
 'beneficiary_resp_ip': 'BENRES_IP',
 'primary_payer_ip': 'PPPYMT_IP',
 'medicare_reimb_op': 'MEDREIMB_OP',
 'beneficiary_resp_op': 'BENRES_OP',
 'primary_payer_op': 'PPPYMT_OP',
 'medicare_reimb_car': 'MEDRE

In [13]:
def age_in_years(bdate_yyyymmdd: int, ref_year: int) -> int:
    y = bdate_yyyymmdd // 10_000
    m = (bdate_yyyymmdd // 100) % 100

    return (ref_year - y) - (m - 1) / 12

In [14]:
df_beneficiaries_2008['age'] = df_beneficiaries_2008[['Year', CB.date_birth]].apply(
    lambda r: age_in_years(r[CB.date_birth], r['Year']), axis=1)

In [15]:
df_beneficiaries_2008[[CB.date_birth, 'age']]

,BENE_BIRTH_DT,age
0,19230501,84.666667
1,19430101,65.000000
2,19360901,71.333333
3,19410601,66.583333
4,19360801,71.416667
...,...,...
116347,19430701,64.500000
116348,19160601,91.583333
116349,19390401,68.750000
116350,19110701,96.500000


In [16]:
binary_characteristics = [c for c in CB.__dict__.keys() if c.startswith('cc')]
binary_characteristics

['cc_esrd_indicator',
 'cc_alzheimer',
 'cc_chf',
 'cc_chronic_kidney',
 'cc_cancer',
 'cc_copd',
 'cc_depression',
 'cc_diabetes',
 'cc_ischemic_heart',
 'cc_osteoporosis',
 'cc_ra_oa',
 'cc_stroke_tia']

In [18]:
df_beneficiaries_2008[[CB[c] for c in binary_characteristics]].describe()

,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA
count,116352.000000,116352.000000,116352.000000,116352.000000,116352.000000,116352.000000,116352.000000,116352.000000,116352.000000,116352.000000,116352.000000
mean,1.807395,1.715046,1.839401,1.936271,1.864695,1.786510,1.621322,1.579363,1.826587,1.846019,1.955110
std,0.394347,0.451395,0.367162,0.244271,0.342050,0.409773,0.485060,0.493664,0.378606,0.360932,0.207063
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [21]:
df_beneficiaries_2008[CB.cc_esrd_indicator].value_counts()

BENE_ESRD_IND
0    108091
Y      8261
Name: count, dtype: int64

In [22]:
medicare_payment = [CB.medicare_reimb_car, CB.medicare_reimb_ip, CB.medicare_reimb_op]
medicare_payment

['MEDREIMB_CAR', 'MEDREIMB_IP', 'MEDREIMB_OP']

In [23]:
df_beneficiaries_2008[CB.state_code].value_counts()

SP_STATE_CODE
5     10224
10     7745
45     6703
33     6510
39     5199
36     4329
14     4277
23     4012
34     3935
31     3176
11     3012
49     2895
44     2803
22     2587
1      2570
15     2497
26     2378
52     2371
3      2325
50     2282
6      2012
24     1982
18     1864
4      1851
42     1848
21     1837
38     1725
19     1715
37     1663
54     1628
7      1473
25     1331
16     1317
51     1152
17     1142
32     1119
46      889
29      842
28      713
20      692
13      675
41      674
30      644
12      544
53      501
8       493
27      429
35      420
43      419
47      374
9       310
2       244
Name: count, dtype: int64

In [24]:
df_beneficiaries_2008[CB.race].value_counts()

BENE_RACE_CD
1    96349
2    12343
3     4931
5     2729
Name: count, dtype: int64

In [ ]:
numerical = ['months_partA', 'months_partB', 'months_hmo', 
             'months_partD']

df_beneficiaries_2008[[C